In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Mapa Cultural do Ceará - Projetos
* Tipo 
* Projeto Pai 
* Nome pessoal, Social, Artístico ou Profissional
* Período de execução do projeto
* Publicado Por 
* Responsáveis
* Descrição do projeot
* Site
* Valor
* Contra Partida
* Área de atuação
* Municípios Contemplados
* Público Alvo
* Tags

## Função para o Scrap de uma página com essa estrutura

In [3]:
def scrap_page(link):
    
    # este headers é um parâmetro necessário para não ter o acesso negado em alguns sites
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'}
    
    # requisitar e salvar a page usando requests
    page = requests.get(link, headers=headers) 
    # criação do objeto soup da page que serve de base para o uso da bilioteca
    soup = BeautifulSoup(page.content, 'html5lib') 
    
    
    
    # bloco para scrapear o tipo e salvar em uma lista tipo_projetos criada anteriormente
    try:
        tipo = soup.find_all(class_="entity-type project-type")[0].get_text().split()[0]
    except:
        tipo = None
        
    tipo_projetos.append(tipo)
    
    # bloco para scrapear o titulo e salvar em uma lista titulos criada anteriormente
    try:
        titulo = " ".join(soup.find_all('h2')[0].get_text().split())
    except:
        titulo = None
        
    titulos.append(titulo)
    
    # bloco para scrapear o publicante e salvar em uma lista publicantes criada anteriormente  
    try:
        publicante = soup.find_all(class_="js-owner-name")[0].get_text()
    except:
        publicante = None
    publicantes.append(publicante)
    
    # bloco para scrapear a descrição do publicante e salvar em uma lista desc_publicantes criada anteriormente     
    try:
        desc_publicante = soup.find_all(class_="owner-description js-owner-description")[0].get_text()

    except:
        desc_publicante = None
    desc_publicantes.append(desc_publicante)
    
    # bloco para scrapear o valor e salvar em uma lista valores criada anteriormente     
    try:
        valor = soup.find_all(class_="servico")[0].find_all("span")[1].get_text().split()[1]
        valor = int(valor.replace(".","").replace(",00",""))
    except:
        valor = None
    valores.append(valor)

    # bloco para scrapear a descrição do projeto e salvar em uma lista desc_projetos criada anteriormente     
    try:
        desc_projeto = soup.find_all(class_="descricao js-editable")[0].get_text()
    except:
        desc_projeto = None
    desc_projetos.append(desc_projeto)
     
    # bloco para scrapear a contrapartida e salvar em uma lista contrapartidas criada anteriormente     
    try:
        contrapartida  = soup.findAll('span',{"data-original-title":"ContraPartida"})[0].get_text()
    except:
        contrapartida = None
    contrapartidas.append(contrapartida)

    # bloco para scrapear a sidebar a esquerda da página
    # e salvar as Áreas de atuação, os Municípios Contemplados, os Públlicos Alvo e as tags
    # nas listas lista_areas, lista_municipios, lista_alvo e lista_tags criadas anteriormente
    try:
        left_bar = soup.find_all(class_="sidebar-left sidebar project")[0].find_all(class_ = 'widget')
    
        areas = []
        municipios = []
        alvo = []
        tags = []
        
        for widget in left_bar:
    
            tipo = " ".join(widget.find_all("h3")[0].get_text().split())
            
            if tipo == "Área de Atuação":

                for tag in widget.find_all(class_='tag tag-project'):
                    areas.append(" ".join(tag.get_text().split()))

            if tipo == "Municípios Contemplados":

                for tag in widget.find_all(class_='tag tag-project'):
                    municipios.append(" ".join(tag.get_text().split()))
                    
            if tipo == "Público-Alvo":

                for tag in widget.find_all(class_='tag tag-project'):
                    alvo.append(" ".join(tag.get_text().split()))
                    
            if tipo == "Tags":

                for tag in widget.find_all(class_='tag tag-project'):
                    tags.append(" ".join(tag.get_text().split()))
        
        lista_areas.append(areas)
        lista_municipios.append(municipios)
        lista_alvo.append(alvo)
        lista_tags.append(tags)
        
    except:
        lista_areas.append(None)
        lista_municipios.append(None)
        lista_alvo.append(None)
        lista_tags.append(None)     

In [4]:
# Listas para receber os dados

tipo_projetos = []
titulos = []
publicantes = []
desc_publicantes = []
valores = []
desc_projetos = []
contrapartidas = []
lista_areas = []
lista_municipios = []
lista_alvo = []
lista_tags = []

## Scrap das páginas

In [6]:
lista_pages1 = [str("https://mapacultural.secult.ce.gov.br/projeto/{}/").format(i) for i in range(3500,4230)]

for link in lista_pages1:
    scrap_page(link)

In [9]:
lista_pages2 = [str("https://mapacultural.secult.ce.gov.br/projeto/{}/").format(i) for i in range(3000,3500)]

for link in lista_pages2:
    scrap_page(link)

In [11]:
lista_pages3 = [str("https://mapacultural.secult.ce.gov.br/projeto/{}/").format(i) for i in range(2000,3000)]

for link in lista_pages3:
    scrap_page(link)

In [12]:
lista_pages4 = [str("https://mapacultural.secult.ce.gov.br/projeto/{}/").format(i) for i in range(1000,2000)]

for link in lista_pages4:
    scrap_page(link)

In [13]:
lista_pages5 = [str("https://mapacultural.secult.ce.gov.br/projeto/{}/").format(i) for i in range(345,1000)]

for link in lista_pages5:
    scrap_page(link)

## Criar um DataFrame com os dados obtidos

In [21]:
# criando dataframe com as listas
df = pd.DataFrame({'Tipo':tipo_projetos, 'Titulo':titulos, 'Publicante':publicantes,
                  'Desc_Publicante':desc_publicantes, 'Valor':valores,
                   'Desc_Projeto':desc_projetos, 'Contrapartida':contrapartidas,
                  'Area_Atuação':lista_areas, 'Municipios_Contemplados':lista_municipios,
                   'Publico_Alvo':lista_alvo, 'Tags':lista_tags})


In [22]:
df = df.dropna(subset=["Titulo"])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2409 entries, 0 to 3884
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Tipo                     2395 non-null   object 
 1   Titulo                   2409 non-null   object 
 2   Publicante               2395 non-null   object 
 3   Desc_Publicante          2395 non-null   object 
 4   Valor                    22 non-null     float64
 5   Desc_Projeto             1253 non-null   object 
 6   Contrapartida            389 non-null    object 
 7   Area_Atuação             2395 non-null   object 
 8   Municipios_Contemplados  2395 non-null   object 
 9   Publico_Alvo             2395 non-null   object 
 10  Tags                     2395 non-null   object 
dtypes: float64(1), object(10)
memory usage: 225.8+ KB


## Exportar os dados

In [26]:
# criando csv
df.to_csv('projetos_mapa_cultural_ce.csv')


In [27]:
df.head()

,Tipo,Titulo,Publicante,Desc_Publicante,Valor,Desc_Projeto,Contrapartida,Area_Atuação,Municipios_Contemplados,Publico_Alvo,Tags
0,Edital,Confessar é um verbo nu,Nágila de Sousa Freitas,Nágila de Sousa Freitas é filha dos agricultor...,NaN,None,None,[Audiovisual],[],[],[]
1,Edital,EDITAL DE CREDENCIAMENTO Nº 006/2021 – SECULT,Secretaria de Cultura de Juazeiro do Norte,Secretaria de Municipal de Cultura de Juazeiro...,NaN,TESTE,None,[],[],[],[]
7,Edital,O Telefone,ObraboMaui,Artista & produtor musical independente,NaN,None,None,[Audiovisual],[],[],[]
8,Jornada,Caminhos Possíveis: PI EXPERIMENTAL e a mobili...,Ariane de Almeida Mendes,Pesquisadora inter-relacionada e gestora de pr...,NaN,O PI EXPERIMENTAL é um laboratório que reúne p...,None,"[Arquitetura-Urbanismo, Artes integradas, Comu...",[Fortaleza],[Mulheres],[]
15,Sarau,Quarentena Literária,Maria de Fátima Santos Souto,"Professora, Coordenadora de Biblioteca Pública...",NaN,None,None,[Literatura],[Trairi],[Crianças],[]
